In [0]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow2/semantic_parsing/tree_slu/main')

In [2]:
!pip install tensorflow-gpu==2.0.0-alpha0

     |████████████████████████████████| 332.1MB 54kB/s 
     |████████████████████████████████| 419kB 45.8MB/s 
     |████████████████████████████████| 61kB 30.0MB/s 
     |████████████████████████████████| 3.0MB 46.2MB/s 


In [3]:
import tensorflow as tf
import numpy as np
import pprint
import logging
import time
import nltk
import os

from pathlib import Path

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 2.0.0-alpha0
GPU Enabled: True


In [0]:
# stream data from text files
def data_generator(f_path, params):
  with open(f_path) as f:
    print('Reading', f_path)
    for line in f:
      text_raw, text_tokenized, label = line.split('\t')
      text_tokenized = text_tokenized.lower().split()
      label = label.replace('[', '[ ').lower().split()
      source = [params['tgt2idx'].get(w, len(params['tgt2idx'])) for w in text_tokenized]
      target = [params['tgt2idx'].get(w, len(params['tgt2idx'])) for w in label]
      target_in = [1] + target
      target_out = target + [2]
      yield (source, target_in, target_out)

In [0]:
def dataset(is_training, params):
  _shapes = ([None], [None], [None])
  _types = (tf.int32, tf.int32, tf.int32)
  _pads = (0, 0, 0)
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['train_samples'])
    ds = ds.padded_batch(params['train_batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['eval_batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [0]:
def get_timing_signal_1d(length,
                         channels,
                         min_timescale=1.0,
                         max_timescale=1.0e4,
                         start_index=0):
  to_float = lambda x: tf.cast(x, tf.float32)
  position = to_float(tf.range(length) + start_index)
  num_timescales = channels // 2
  log_timescale_increment = (
      tf.math.log(float(max_timescale) / float(min_timescale)) /
      tf.maximum(to_float(num_timescales) - 1, 1))
  inv_timescales = min_timescale * tf.exp(
      to_float(tf.range(num_timescales)) * -log_timescale_increment)
  scaled_time = tf.expand_dims(position, 1) * tf.expand_dims(inv_timescales, 0)
  signal = tf.concat([tf.sin(scaled_time), tf.cos(scaled_time)], axis=1)
  signal = tf.pad(signal, [[0, 0], [0, tf.mod(channels, 2)]])
  signal = tf.reshape(signal, [1, length, channels])
  return signal

In [0]:
class LayerNorm(tf.keras.layers.Layer):
  def __init__(self, params):
    super().__init__()
    self._epsilon = params['epsilon']
    self._hidden_units = params['global_units']
  
  def build(self, input_shape):
    self.scale = self.add_weight(name='scale',
                                 shape=[self._hidden_units],
                                 initializer=tf.ones_initializer(),
                                 trainable=True)
    self.bias = self.add_weight(name='bias',
                                shape=[self._hidden_units],
                                initializer=tf.zeros_initializer(),
                                trainable=True)
    super().build(input_shape)
  
  def call(self, inputs):
    mean, variance = tf.nn.moments(inputs, [-1], keepdims=True)
    norm_x = (inputs - mean) * tf.math.rsqrt(variance + self._epsilon)
    return norm_x * self.scale + self.bias
  
  def compute_output_shape(self, input_shape):
    return input_shape

In [0]:
class PointwiseFFNBlock(tf.keras.Model):
  def __init__(self, params):
    super().__init__()
    self.layer_norm = LayerNorm(params)
    self.block_dropout = tf.keras.layers.Dropout(params['dropout_rate'])

    self.filter = tf.keras.layers.Dense(params['multiplier']*params['global_units'], tf.nn.relu, name='filter')
    self.dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    self.linear = tf.keras.layers.Dense(params['global_units'], name='linear')
  
  
  def call(self, inputs, training=False):
    x = self.layer_norm(inputs)
    x = self.forward(x, training=training)
    x = self.block_dropout(x, training=training)
    x += inputs
    return x
    
  
  def forward(self, x, training):
    return self.linear(self.dropout(self.filter(x), training=training))

In [0]:
class SelfAttentionBlock(tf.keras.Model):
  def __init__(self, params, is_bidirectional):
    super().__init__()
    
    self.layer_norm = LayerNorm(params)
    self.block_dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    
    self.qkv_linear = tf.keras.layers.Dense(3 * params['global_units'], name='qkv_linear')
    self.dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    self.out_linear = tf.keras.layers.Dense(params['global_units'], name='out_linear')
    
    self._is_bidirectional = is_bidirectional
    self._num_heads = params['num_heads']
  
  
  def call(self, inputs, training=False):
    inputs, masks = inputs
    x = self.layer_norm(inputs)
    x = self.forward((x, masks), training=training)
    x = self.block_dropout(x, training=training)
    x += inputs
    return x
    
  
  def forward(self, inputs, training):
    x, masks = inputs
    timesteps = tf.shape(x)[1]
    
    q_k_v = self.qkv_linear(x)
    q, k, v = tf.split(q_k_v, 3, axis=-1)
    
    if self._num_heads > 1:
      q = tf.concat(tf.split(q, self._num_heads, axis=2), axis=0)                        
      k = tf.concat(tf.split(k, self._num_heads, axis=2), axis=0)                        
      v = tf.concat(tf.split(v, self._num_heads, axis=2), axis=0)
    
    align = tf.matmul(q, k, transpose_b=True)
    align *= tf.math.rsqrt(tf.cast(k.shape[-1], tf.float32))
    
    if (masks is not None) or (not self._is_bidirectional):
      paddings = tf.fill(tf.shape(align), float('-inf'))
    
    if masks is not None:
      c_masks = tf.tile(masks, [params['num_heads'], 1])
      c_masks = tf.tile(tf.expand_dims(c_masks, 1), [1, timesteps, 1])
      align = tf.where(tf.equal(c_masks, 0), paddings, align)
    
    if not self._is_bidirectional:
      lower_tri = tf.ones((timesteps, timesteps))                                       
      lower_tri = tf.linalg.LinearOperatorLowerTriangular(lower_tri).to_dense()      
      t_masks = tf.tile(tf.expand_dims(lower_tri, 0), [tf.shape(align)[0], 1, 1])     
      align = tf.where(tf.equal(t_masks, 0), paddings, align)
    
    align = tf.nn.softmax(align)
    align = self.dropout(align, training=training)
    
    if masks is not None:
      q_masks = tf.tile(masks, [params['num_heads'], 1])
      q_masks = tf.tile(tf.expand_dims(q_masks, 2), [1, 1, timesteps])
      align *= tf.cast(q_masks, tf.float32)
    
    x = tf.matmul(align, v)
    if self._num_heads > 1:
      x = tf.concat(tf.split(x, self._num_heads, axis=0), axis=2)
    x = self.out_linear(x)
    
    return x

In [0]:
class MutualAttentionBlock(tf.keras.Model):
  def __init__(self, params):
    super().__init__()
    self.layer_norm = LayerNorm(params)
    self.block_dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    
    self.q_linear = tf.keras.layers.Dense(params['global_units'], name='q_linear')
    self.kv_linear = tf.keras.layers.Dense(2*params['global_units'], name='kv_linear')
    self.dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    self.out_linear = tf.keras.layers.Dense(params['global_units'], name='out_linear')
    
    self._num_heads = params['num_heads']
  
  
  def call(self, inputs, training=False):
    inputs, mask_dec, encoded, mask_enc = inputs
    x = self.layer_norm(inputs)
    x = self.forward((x, mask_dec, encoded, mask_enc), training=training)
    x = self.block_dropout(x, training=training)
    x += inputs
    return x
    
  
  def forward(self, inputs, training):
    query, mask_query, context, mask_context = inputs
    time_query, time_context = tf.shape(query)[1], tf.shape(context)[1]
    
    q = self.q_linear(query)
    k_v = self.kv_linear(context)
    k, v = tf.split(k_v, 2, axis=-1)
    
    if self._num_heads > 1:
      q = tf.concat(tf.split(q, self._num_heads, axis=2), axis=0)                        
      k = tf.concat(tf.split(k, self._num_heads, axis=2), axis=0)                        
      v = tf.concat(tf.split(v, self._num_heads, axis=2), axis=0)
    
    align = tf.matmul(q, k, transpose_b=True)
    align *= tf.math.rsqrt(tf.cast(k.shape[-1], tf.float32))
    
    paddings = tf.fill(tf.shape(align), float('-inf'))
    context_masks = tf.tile(mask_context, [self._num_heads, 1])
    context_masks = tf.tile(tf.expand_dims(context_masks, 1), [1, time_query, 1])
    align = tf.where(tf.equal(context_masks, 0), paddings, align)
    
    align = tf.nn.softmax(align)
    align = self.dropout(align, training=training)
    
    query_masks = tf.tile(mask_query, [self._num_heads, 1])
    query_masks = tf.tile(tf.expand_dims(query_masks, 2), [1, 1, time_context])
    align *= tf.cast(query_masks, tf.float32)
    
    x = tf.matmul(align, v)
    if self._num_heads > 1:
      x = tf.concat(tf.split(x, self._num_heads, axis=0), axis=2)
    x = self.out_linear(x)
    
    return x

In [0]:
class EncoderLayer(tf.keras.Model):
  def __init__(self, params, name):
    super().__init__(name=name)
    self.self_attention = SelfAttentionBlock(params, is_bidirectional=True)
    self.pointwise_ffn = PointwiseFFNBlock(params)
  
  
  def call(self, inputs, training=False):
    x, mask = inputs
    x = self.self_attention((x, mask), training=training)
    x = self.pointwise_ffn(x, training=training)
    return x

In [0]:
class DecoderLayer(tf.keras.Model):
  def __init__(self, params, name):
    super().__init__(name=name)
    self.self_attention = SelfAttentionBlock(params, is_bidirectional=False)
    self.mutual_attention = MutualAttentionBlock(params)
    self.pointwise_ffn = PointwiseFFNBlock(params)
  
  
  def call(self, inputs, training=False):
    decoded, mask_dec, encoded, mask_enc = inputs
    decoded = self.self_attention((decoded, mask_dec), training=training)
    decoded = self.mutual_attention((decoded, mask_dec, encoded, mask_enc), training=training)
    decoded = self.pointwise_ffn(decoded, training=training)
    return decoded

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, params):
    super().__init__()
    self.params = params
    self.embedding = tf.keras.layers.Embedding(len(params['tgt2idx']), params['global_units'],
      embeddings_initializer=tf.initializers.RandomNormal(stddev=params['global_units'] ** -0.5))
    self.input_dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    self.encodes = [EncoderLayer(params, name='enc_layer_{}'.format(i+1)) for i in range(params['num_layers'])]
    
  
  def call(self, inputs, training=False):
    input_enc = inputs
    
    mask_enc = tf.sign(input_enc)
    
    input_enc = self.embedding(input_enc)
    
    encoded = self.input_transform(input_enc, training=training)
    
    for layer in self.encodes:
      encoded = layer((encoded, mask_enc), training=training)
    
    return encoded
  
  
  def input_transform(self, x, training):
    if self.params['is_embedding_scaled']:
      x *= tf.sqrt(tf.cast(self.params['global_units'], tf.float32))
    x += get_timing_signal_1d(tf.shape(x)[1], self.params['global_units'])
    x = self.input_dropout(x, training=training)
    return x

In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, params, tied_embedding):
    super().__init__()
    self.params = params
    self.embedding = tied_embedding
    self.input_dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    self.decodes = [DecoderLayer(params, name='dec_layer_{}'.format(i+1)) for i in range(params['num_layers'])]
    self.out_bias = self.add_weight(name='out_bias', shape=[len(params['tgt2idx'])])
    
  
  def call(self, inputs, training=False):
    input_dec, memory, memory_mask = inputs
    
    mask_dec = tf.sign(input_dec)
    
    input_dec = self.embedding(input_dec)
    
    decoded = self.input_transform(input_dec, training=training)
    
    for layer in self.decodes:
      decoded = layer((decoded, mask_dec, memory, memory_mask), training=training)
    
    logits = self.tied_output(decoded)
    return logits
  
  
  def input_transform(self, x, training):
    if self.params['is_embedding_scaled']:
      x *= tf.sqrt(tf.cast(self.params['global_units'], tf.float32))
    x += get_timing_signal_1d(tf.shape(x)[1], self.params['global_units'])
    x = self.input_dropout(x, training=training)
    return x
  
  
  def tied_output(self, decoded):
    axis_1, axis_2  = tf.shape(decoded)[0], tf.shape(decoded)[1]
    decoded = tf.reshape(decoded, (axis_1*axis_2, params['global_units']))
    logits = tf.matmul(decoded, self.embedding.embeddings, transpose_b=True)
    logits = tf.reshape(logits, (axis_1, axis_2, len(self.params['tgt2idx'])))
    logits = tf.nn.bias_add(logits, self.out_bias)
    return logits

In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip()
      word2idx[line] = i
  return word2idx

In [0]:
def is_descending(history: list) -> bool:
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True

In [0]:
params = {
    'train_path': '../data/train.tsv',
    'test_path': '../data/test.tsv',
    'vocab_src_path': '../vocab/source.txt',
    'vocab_tgt_path': '../vocab/target.txt',
    'model_path': '../model/',
    'num_layers': 3,
    'dropout_rate': 0.2,
    'global_units': 300,
    'num_heads': 4,
    'multiplier': 2,
    'epsilon': 1e-6,
    'is_embedding_scaled': True,
    'max_decode_len': 50,
    'lr': 4e-4,
    'train_samples': 31279,
    'train_batch_size': 32,
    'eval_samples': 9042,
    'eval_batch_size': 300,
    'num_patience': 5,
}

In [0]:
params['tgt2idx'] = get_vocab(params['vocab_tgt_path'])
params['idx2tgt'] = {idx: tgt for tgt, idx in params['tgt2idx'].items()}

In [19]:
Path(params['model_path']).mkdir(exist_ok=True)

encoder = Encoder(params)
encoder.build((None, None))
pprint.pprint([(v.name, v.shape) for v in encoder.trainable_variables])

decoder = Decoder(params, encoder.embedding)
decoder.build([[None, None], [None, None, params['global_units']], [None, None]])
pprint.pprint([(v.name, v.shape) for v in decoder.trainable_variables])

[('embedding/embeddings:0', TensorShape([8691, 300])),
 ('enc_layer_1/self_attention_block/layer_norm/scale:0', TensorShape([300])),
 ('enc_layer_1/self_attention_block/layer_norm/bias:0', TensorShape([300])),
 ('enc_layer_1/self_attention_block/qkv_linear/kernel:0',
  TensorShape([300, 900])),
 ('enc_layer_1/self_attention_block/qkv_linear/bias:0', TensorShape([900])),
 ('enc_layer_1/self_attention_block/out_linear/kernel:0',
  TensorShape([300, 300])),
 ('enc_layer_1/self_attention_block/out_linear/bias:0', TensorShape([300])),
 ('enc_layer_1/pointwise_ffn_block/layer_norm_1/scale:0', TensorShape([300])),
 ('enc_layer_1/pointwise_ffn_block/layer_norm_1/bias:0', TensorShape([300])),
 ('enc_layer_1/pointwise_ffn_block/filter/kernel:0', TensorShape([300, 600])),
 ('enc_layer_1/pointwise_ffn_block/filter/bias:0', TensorShape([600])),
 ('enc_layer_1/pointwise_ffn_block/linear/kernel:0', TensorShape([600, 300])),
 ('enc_layer_1/pointwise_ffn_block/linear/bias:0', TensorShape([300])),
 ('en

In [20]:
decay_lr = tf.optimizers.schedules.ExponentialDecay(params['lr'], 1000, 0.96)
optim = tf.optimizers.Adam(params['lr'])
global_step = 7824
encoder.load_weights(os.path.join(params['model_path'], 'encoder_{}'.format(global_step)))
decoder.load_weights(os.path.join(params['model_path'], 'decoder_{}'.format(global_step)))

In [0]:
history_acc = []
best_acc = .0

In [0]:
t0 = time.time()
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)

In [0]:
def minimal_test(encoder, decoder, params):
  test_str = ['what', 'times', 'are', 'the', 'nutcracker', 'show', 'playing', 'near', 'me']
  test_arr = tf.convert_to_tensor([[params['tgt2idx'][w] for w in test_str]])
  generated = tf.convert_to_tensor([[1]])
  memory = encoder(test_arr, training=False)
  memory_mask = tf.sign(test_arr)
  
  for i in range(params['max_decode_len']):
    logits = decoder((generated, memory, memory_mask), training=False)
    ids = tf.argmax(logits[:, i, :], axis=-1, output_type=tf.int32)
    ids = tf.expand_dims(ids, 1)
    generated = tf.concat((generated, ids), axis=1)
    if np.asscalar(ids.numpy()) == 2:
      break
  print('-'*12)
  print('minimal test')
  print('utterance:', ' '.join(test_str))
  parsed = ' '.join([params['idx2tgt'][idx] for idx in generated[0].numpy()[1:-1]])
  print('parsed:', parsed)
  print()
  try:
    nltk.tree.Tree.fromstring(parsed.replace('[ ', '(').replace(' ]', ')')).pretty_print()
  except:
    pass
  print('-'*12)

In [0]:
def is_descending(history: list) -> bool:
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True  

In [25]:
while True:
  # TRAINING
  is_training = True
  for (source, target_in, target_out) in dataset(is_training=is_training, params=params):
    with tf.GradientTape() as tape:
      memory = encoder(source, training=is_training)
      logits = decoder((target_in, memory, tf.sign(source)), training=is_training)
      loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=target_out, logits=logits)
      weights = tf.cast(tf.sign(target_out), tf.float32)
      loss = tf.reduce_sum(loss * weights) / tf.reduce_sum(weights)
      
    optim.lr.assign(decay_lr(global_step))
    variables = encoder.trainable_variables + decoder.trainable_variables
    grads = tape.gradient(loss, variables)
    optim.apply_gradients(zip(grads, variables))
    
    if global_step % 50 == 0:
      logger.info("Step {} | Loss: {:.4f} | Spent: {:.1f} secs | LR: {:.6f}".format(
          global_step, loss.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
      t0 = time.time()
    
    global_step += 1
  
  # EVALUATION
  minimal_test(encoder, decoder, params)
  m = tf.keras.metrics.Mean()
  is_training=False
  
  for i, (source, target_in, target_out) in enumerate(dataset(is_training=is_training, params=params)):
    generated = tf.ones((source.shape[0], 1), tf.int32)
    memory = encoder(source, training=is_training)
    memory_mask = tf.sign(source)
    
    for j in range(target_out.shape[1]):
      logits = decoder((generated, memory, memory_mask), training=is_training)
      ids = tf.argmax(logits[:, j, :], axis=-1, output_type=tf.int32)
      ids = tf.expand_dims(ids, 1)
      generated = tf.concat((generated, ids), axis=1)

    seq_lens = tf.argmax(tf.cast(tf.equal(target_out, 2), tf.int32), axis=1)
    for pred, tgt, seq_len in zip(generated.numpy(), target_out.numpy(), seq_lens.numpy()):
      pred = pred[1:][:seq_len+1]
      tgt = tgt[:seq_len+1]
      matched = np.all(pred == tgt)
      m.update_state(int(matched))
    print("Testing [{}/{}]".format(i, params['eval_samples']//params['eval_batch_size']))
  
  acc = m.result().numpy()
  logger.info("Evaluation: Testing Exact Match Accuracy: {:.3f}".format(acc))
  history_acc.append(acc)

  if acc > best_acc:
    best_acc = acc
    encoder.save_weights(os.path.join(params['model_path'], 'encoder_{}'.format(global_step)))
    decoder.save_weights(os.path.join(params['model_path'], 'decoder_{}'.format(global_step)))
  logger.info("Best Accuracy: {:.3f}".format(best_acc))

  if len(history_acc) > params['num_patience'] and is_descending(history_acc):
    logger.info("Testing Accuracy not improved over {} epochs, Early Stop".format(params['num_patience']))
    break

W0510 00:07:14.315335 139702514296704 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:410: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


Reading ../data/train.tsv


I0510 00:07:42.748516 139702514296704 interactiveshell.py:2882] Step 7850 | Loss: 0.1614 | Spent: 28.6 secs | LR: 0.000290
I0510 00:08:21.117797 139702514296704 interactiveshell.py:2882] Step 7900 | Loss: 0.1047 | Spent: 38.4 secs | LR: 0.000290
I0510 00:09:00.469114 139702514296704 interactiveshell.py:2882] Step 7950 | Loss: 0.0837 | Spent: 39.3 secs | LR: 0.000289
I0510 00:09:38.729655 139702514296704 interactiveshell.py:2882] Step 8000 | Loss: 0.1150 | Spent: 38.3 secs | LR: 0.000289
I0510 00:10:18.183177 139702514296704 interactiveshell.py:2882] Step 8050 | Loss: 0.1483 | Spent: 39.5 secs | LR: 0.000288
I0510 00:10:56.310026 139702514296704 interactiveshell.py:2882] Step 8100 | Loss: 0.1833 | Spent: 38.1 secs | LR: 0.000287
I0510 00:11:35.126235 139702514296704 interactiveshell.py:2882] Step 8150 | Loss: 0.0992 | Spent: 38.8 secs | LR: 0.000287
I0510 00:12:13.351990 139702514296704 interactiveshell.py:2882] Step 8200 | Loss: 0.0767 | Spent: 38.2 secs | LR: 0.000286
I0510 00:12:52.2

------------
minimal test
utterance: what times are the nutcracker show playing near me
parsed: [ in:get_event what times are [ sl:category_event the nutcracker show ] playing [ sl:location [ in:get_location [ sl:search_radius near ] [ sl:location_user me ] ] ] ]

                       in:get_event                                                                        
  __________________________|_____________________________________________________                          
 |     |    |     |                        |                                 sl:location                   
 |     |    |     |                        |                                      |                         
 |     |    |     |                        |                               in:get_location                 
 |     |    |     |                        |                      ________________|_______________          
 |     |    |     |                 sl:category_even      sl:search_radius          

I0510 00:32:41.630346 139702514296704 interactiveshell.py:2882] Evaluation: Testing Exact Match Accuracy: 0.641
I0510 00:32:41.826500 139702514296704 interactiveshell.py:2882] Best Accuracy: 0.641


...

------------
minimal test
utterance: what times are the nutcracker show playing near me
parsed: [ in:get_event what times are [ sl:category_event the nutcracker show playing ] [ sl:location [ in:get_location [ sl:search_radius near ] [ sl:location_user me ] ] ] ]

                     in:get_event                                                                 
  ________________________|______________________________________________                          
 |     |    |             |                                         sl:location                   
 |     |    |             |                                              |                         
 |     |    |             |                                       in:get_location                 
 |     |    |             |                              ________________|_______________          
 |     |    |      sl:category_even              sl:search_radius                 sl:location_user
 |     |    |             t            

I0510 09:43:11.707387 139702514296704 interactiveshell.py:2882] Evaluation: Testing Exact Match Accuracy: 0.702
I0510 09:43:11.708534 139702514296704 interactiveshell.py:2882] Best Accuracy: 0.704


Testing [30/30]
Reading ../data/train.tsv


I0510 09:43:41.599024 139702514296704 interactiveshell.py:2882] Step 30350 | Loss: 0.0260 | Spent: 799.2 secs | LR: 0.000116
I0510 09:44:20.686997 139702514296704 interactiveshell.py:2882] Step 30400 | Loss: 0.0283 | Spent: 39.1 secs | LR: 0.000116
I0510 09:44:58.045119 139702514296704 interactiveshell.py:2882] Step 30450 | Loss: 0.0514 | Spent: 37.4 secs | LR: 0.000115
I0510 09:45:36.002571 139702514296704 interactiveshell.py:2882] Step 30500 | Loss: 0.0317 | Spent: 38.0 secs | LR: 0.000115
I0510 09:46:12.734453 139702514296704 interactiveshell.py:2882] Step 30550 | Loss: 0.0471 | Spent: 36.7 secs | LR: 0.000115
I0510 09:46:50.876873 139702514296704 interactiveshell.py:2882] Step 30600 | Loss: 0.0305 | Spent: 38.1 secs | LR: 0.000115
I0510 09:47:27.595255 139702514296704 interactiveshell.py:2882] Step 30650 | Loss: 0.0138 | Spent: 36.7 secs | LR: 0.000114
I0510 09:48:05.501924 139702514296704 interactiveshell.py:2882] Step 30700 | Loss: 0.0348 | Spent: 37.9 secs | LR: 0.000114
I0510 0

KeyboardInterrupt: ignored